# Import package

In [2]:
import os
import pandas as pd
from root_numpy import root2array 

Welcome to JupyROOT 6.20/04


# Load root file

In [62]:
root_file = "./SVJ_pythia/delphes_nolhe_Z1000_rinv_03.root" #your root file path

event = root2array(root_file, "Delphes;1", branches=[
"Particle.Status", "Particle.M1", "Particle.M2", "Particle.D1", "Particle.D2",
"Particle.PID", "Particle.PT",
"Particle.Eta","Particle.Phi", "Particle.Mass"], start=0, stop=2000, step=None)
# in this example I set 
_Status, _M1, _M2, _D1, _D2, _PID, _PT, _Eta, _Phi, _Mass = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
Labels = ["Status", "M1", "M2", "D1", "D2", "PID", "PT", "Eta", "Phi", "Mass"]



# Show truth record

In [48]:
print("Index", "\t","Status", "\t","M1" "\t","M2" ,"\t","D1", "\t","D2", "\t","PID", "\t\t","PT" "\t","Eta", "\t\t","Phi", "\t\t","Mass")
for j in range(len(event[0][0])):  
    print(j, "\t", event[0][_Status][j],  "\t\t", event[0][_M1][j], "\t", event[0][_M2][j], "\t", event[0][_D1][j], "\t", 
          event[0][_D2][j],  "\t", str(event[0][_PID][j]).ljust(12, ' '), "\t", round(event[0][_PT][j],0),  "\t",
          str(round(event[0][_Eta][j],2)).ljust(12, ' ') , "\t",str(round(event[0][_Phi][j],3)).ljust(12, ' '),  "\t",round(event[0][_Mass][j],3)) 
 

Index 	 Status 	 M1	 M2 	 D1 	 D2 	 PID 		 PT	 Eta 		 Phi 		 Mass
0 	 4 		 -1 	 -1 	 459 	 -1 	 2212         	 0.0 	 999.9        	 0.0          	 0.938
1 	 4 		 -1 	 -1 	 460 	 -1 	 2212         	 0.0 	 -999.9       	 0.0          	 0.938
2 	 21 		 12 	 -1 	 4 	 5 	 1            	 0.0 	 999.9        	 0.0          	 0.0
3 	 21 		 13 	 13 	 4 	 5 	 -1           	 0.0 	 -999.9       	 0.0          	 0.0
4 	 23 		 2 	 3 	 8 	 8 	 4900101      	 417.0 	 1.18         	 0.701        	 10.0
5 	 23 		 2 	 3 	 6 	 7 	 -4900101     	 417.0 	 -1.47        	 -2.44        	 10.0
6 	 51 		 5 	 -1 	 11 	 11 	 -4900101     	 146.0 	 0.03         	 -0.495       	 10.0
7 	 51 		 5 	 -1 	 9 	 10 	 4900021      	 440.0 	 -1.34        	 -2.755       	 0.0
8 	 52 		 4 	 4 	 14 	 14 	 4900101      	 365.0 	 1.18         	 0.701        	 10.0
9 	 51 		 7 	 -1 	 16 	 16 	 4900021      	 334.0 	 -1.1         	 3.104        	 0.0
10 	 51 		 7 	 -1 	 17 	 17 	 4900021      	 205.0 	 -1.26        	 -1.703       	

# Define function to trace particle decay

In [5]:
def find_decayratio(event,n, mode = 'daughter'):
    d1, d2 = -1, -1
    d0 = -1
    ID = 4900101 #Xd PID
    for j in  range(len(event[n][0])):
        if(event[n][5][j]==ID)&(event[n][0][j]==23): #23 : hardest outgoing particles
            d1 = event[n][_D1][j]
            d2 = event[n][_D2][j]
            while((event[n][_PID][d1]==ID)or(event[n][5][d2]==ID) ):
                if(event[n][_PID][d1]==ID):
                    d0 = d1
                    d1 = event[n][_D1][d0]
                    d2 = event[n][_D2][d0]
                    if(event[n][_PID][d2]==ID):
                        d0 = d2
                        d1 = event[n][_D1][d0]
                        d2 = event[n][_D2][d0]
                        
    if mode=='daughter':
        return d1, d2
    if mode=='id':
        return event[n][5][d1], event[n][5][d2]
    
def find_subdecayratio(event, n, j, ID=4900111): 
    d1, d2 = -1, -1
    d0 = -1
    d1 = event[n][_D1][j]
    d2 = event[n][_D2][j]
    if((abs(event[n][_PID][d1])!=ID)&(abs(event[n][_PID][d2])!=ID)):
        d1=0
        d2=0
        return d1, d2
    else:
        while((abs(event[n][_PID][d1])==ID)or(abs(event[n][_PID][d2])==ID) ):
            if(abs(event[n][_PID][d1])==ID):
                d0 = d1
                d1 = event[n][_D1][d0]
                d2 = event[n][_D2][d0]
            if(abs(event[n][_PID][d2])==ID):
                d0 = d2
                d1 = event[n][_D1][d0]
                d2 = event[n][_D2][d0]
        return event[n][_PID][d1], event[n][_PID][d2]
def find_daughterid(event, n, j, ID=4900111):
    sid = event[n][_PID][j]
    if(sid!=ID):
        return 0, 0
    else:
        d1 = event[n][_D1][j]
        d2 = event[n][_D2][j]
        return event[n][_PID][d1], event[n][_PID][d2]

        

# Trace certain particle ID untile it "decay" (daughter1!=daughter2)

In [58]:
n=0
ID=4900113
N = []
for j in  range(len(event[n][0])):
    if(event[n][_PID][j]==ID): #23 : hardest outgoing particles
        print("Self index in event",j)
        m1 = event[n][_M1][j]
        m2 = event[n][_M2][j]
        print("Mother:",m1,m2)
        if (m1==m2):
            print("Mother_id:", event[n][_PID][m1], "nan\n")
        else:
            print("Mother_id:", event[n][_PID][m1], event[n][_PID][m2],"\n")
        d1 = event[n][_D1][j]
        d2 = event[n][_D2][j]
        prevent = 0
        while(d1==d2): 
            d0 = d1
            if(d1==-1):
                print("stable particles\n\n")
                break
            if(prevent >= 100):
                print("infinite loop!")
                prevent=0
                break
            d1 = event[n][_D1][d1]
            d2 = event[n][_D2][d2]
            if(d1!=d2):
                N.append(d0)
            prevent= prevent+1
        if(d1!=-1):
            print("Daughter", d1, d2)
            print("Daughter_id", event[n][_PID][d1], event[n][_PID][d2])
            m1, m2 = event[n][_M1][d1], event[n][_M2][d1]
            print("Daughter1_parents", m1, m2)
            if(m1==m2):
                print("Daughter1_parents_id", event[n][_PID][m1], "nan") 
            else:
                print("Daughter1_parents_id", event[n][_PID][m1], event[n][_PID][m2])
            m1, m2 = event[n][_M1][d2], event[n][_M2][d2]
            print("Daughter2_parents", m1, m2)
            if(m1==m2): 
                print("Daughter2_parents_id", event[n][_PID][m1], "nan","\n")
            else:
                print("Daughter2_parents_id", event[n][_PID][m1], event[n][5][m2],"\n")

Self index in event 636
Mother: 594 635
Mother_id: 4900101 -4900101 

Daughter 1034 1035
Daughter_id -3 3
Daughter1_parents 636 -1
Daughter1_parents_id 4900113 -211
Daughter2_parents 636 -1
Daughter2_parents_id 4900113 -211 

Self index in event 637
Mother: 594 635
Mother_id: 4900101 -4900101 

Daughter 1048 1049
Daughter_id -5 5
Daughter1_parents 637 -1
Daughter1_parents_id 4900113 -211
Daughter2_parents 637 -1
Daughter2_parents_id 4900113 -211 

Self index in event 646
Mother: 594 635
Mother_id: 4900101 -4900101 

Daughter 1084 1085
Daughter_id -4 4
Daughter1_parents 646 -1
Daughter1_parents_id 4900113 -211
Daughter2_parents 646 -1
Daughter2_parents_id 4900113 -211 

Self index in event 648
Mother: 594 635
Mother_id: 4900101 -4900101 

Daughter 1098 1099
Daughter_id -5 5
Daughter1_parents 648 -1
Daughter1_parents_id 4900113 -211
Daughter2_parents 648 -1
Daughter2_parents_id 4900113 -211 

Self index in event 651
Mother: 594 635
Mother_id: 4900101 -4900101 

Daughter 1120 1121
Daughte

# Trace outgoing Xd (4900101) in hardest subprocess 
# And then record the dark meson decay

In [63]:
# 4900111 and 4900113 is unstable dark meson
a113 = []
a111 = []
for n in range(1000):
    a1, a2 = find_decayratio(event,n) # find Xd -> a1 a2
    d1a1 = find_daughterid(event, n, a1, ID=4900111) # check whether a1 is 4900111 and find it's daughter
    d2a1 = find_daughterid(event, n, a2, ID=4900111) # check whether a2 is 4900111 and find it's daughter
    
    d1a2 = find_daughterid(event, n, a1, ID=4900113) # check whether a1 is 4900113 and find it's daughter
    d2a2 = find_daughterid(event, n, a2, ID=4900113) # check whether a2 is 4900113 and find it's daughter

    a111.append(d1a1)
    a111.append(d2a1)
    a113.append(d1a2)
    a113.append(d2a2)

# Calculate the ratio of stable dark meson in above record

In [64]:
#find 4900211 in 4900111 decay record (Note 4900211 is stable dark meson)
r = [sum([1 for i in a111 for j in i if (abs(j)== 4900211)]), sum([1 for i in a111 for j in i if (abs(j)!= 4900211)&(j!=0)])] 
print("The number_4900211/number_others in 4900111 decay particles ", r[0]/sum(r))

The number_4900211/number_others in 4900111 decay particles  0.28363636363636363


In [65]:
#find 4900213 in 4900113 decay record (Note 4900211 is stable dark meson)
r = [sum([1 for i in a113 for j in i if (abs(j)== 4900213)]), sum([1 for i in a113 for j in i if (abs(j)!= 4900213)&(j!=0)])] 
print("The number_4900213/number_others in 4900113 decay particles ", r[0]/sum(r))

The number_4900213/number_others in 4900113 decay particles  0.3028571428571429


# End